In [20]:
import ollama

model = "hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF:IQ4_XS"
info = ollama.show(model).modelinfo
for k, v in info.items():
    print(f"{k}: {v}")




general.architecture: llama
general.basename: Llama-3.2
general.file_type: 30
general.finetune: Instruct
general.languages: ['en', 'de', 'fr', 'it', 'pt', 'hi', 'es', 'th']
general.license: llama3.2
general.parameter_count: 1235814432
general.quantization_version: 2
general.size_label: 1B
general.tags: ['facebook', 'meta', 'pytorch', 'llama', 'llama-3', 'text-generation']
general.type: model
llama.attention.head_count: 32
llama.attention.head_count_kv: 8
llama.attention.key_length: 64
llama.attention.layer_norm_rms_epsilon: 1e-05
llama.attention.value_length: 64
llama.block_count: 16
llama.context_length: 131072
llama.embedding_length: 2048
llama.feed_forward_length: 8192
llama.rope.dimension_count: 64
llama.rope.freq_base: 500000
llama.vocab_size: 128256
quantize.imatrix.chunks_count: 125
quantize.imatrix.dataset: /training_dir/calibration_datav3.txt
quantize.imatrix.entries_count: 112
quantize.imatrix.file: /models_out/Llama-3.2-1B-Instruct-GGUF/Llama-3.2-1B-Instruct.imatrix
tokenize

In [21]:
from ollama import chat

prompt = "Give me a simple recipe for a delicious citrusy cake. Make sure units are in grams when it makes sense. Temperatures should be in C."

response = chat(
    model=model,
    messages=[
        {"role": "user", "content": prompt},
    ],
)
print(response.message.content)

Here's a simple recipe for a delicious citrusy cake that fits your requirements:

**Lemon Sunshine Cake**

Ingredients:

* 250g unsalted butter, softened
* 500g granulated sugar
* 4 large eggs, at room temperature
* 150g freshly squeezed lemon juice (about 6-8 lemons)
* 120g all-purpose flour
* 10g salt
* 2g baking powder
* 1 tsp grated lemon zest

Instructions:

1. Preheat your oven to 180°C (350°F) and grease two 12cm (4.7 inch) round cake pans.
2. In a large mixing bowl, cream the butter until light and fluffy, about 5 minutes. Add the sugar and continue to beat until well combined, about 3-4 minutes.
3. Beat in the eggs one at a time, allowing each egg to fully incorporate before adding the next. Beat in the lemon juice, making sure it's evenly distributed throughout the mixture.
4. In a separate bowl, whisk together the flour, salt, and baking powder. Gradually add this dry mixture to the wet ingredients and mix until just combined.
5. Stir in the grated lemon zest.
6. Divide the 

In [22]:
from ollama import ChatResponse, chat
from pydantic.types import JsonSchemaValue
from typing import Optional, List

system_prompt = "You are a helpful assistant that provides clear and concise answers to the user's needs. Always answer in a JSON format."

def generate(
    prompt: str,
    json_format: Optional[JsonSchemaValue] = None,
    model=model,
    system_prompt=system_prompt,
) -> str:
    response: ChatResponse = chat(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt},
        ],
        options={
            "num_ctx": 4096,
            "num_predict": 1024,
            "top_k": 50,
            "top_p": 0.95,
            "temperature": 0.0,
            "seed": 0,  # this is not needed when temp is 0
            "repeat_penalty": 1.0,  # remain default for json outputs, from experience.
        },
        format=json_format,
        stream=False,
    )
    return response.message.content

# prompt = "give me 5 interesting facts about the universe"
prompt = "Give me a simple recipe for a delicious citrusy cake. Make sure units are in grams when it makes sense. Temperatures should be in C."
print(generate(prompt))

KeyboardInterrupt: 